In [60]:
import requests
from bs4 import BeautifulSoup as bs
import types
import json
import pymysql
import time

db = pymysql.connect('140.138.77.90','knkn','tp6bjo4u;6','knkn',use_unicode=True, charset="utf8mb4")
cursor = db.cursor()

owner_id="1593077"
user_url = "https://stackoverflow.com/users/" + owner_id
user_page = bs(requests.get(user_url).text,'html.parser')
    
reputation = user_page.find('div',class_='reputation').text.split(' r')[0]#可能大於一千，就會有,不能直接轉型
reputation = int(reputation.strip().replace(',',''))
print(reputation)

24124


In [61]:
answers = int(user_page.find('div',class_='stat answers col-3').text.split('a')[0].strip().replace(',',''))
print(answers)

questions = int(user_page.find('div',class_='stat questions col-3').text.split('q')[0].strip().replace(',',''))
print(questions)

reached = user_page.find('div',class_='stat people-helped col-5').text.split('p')[0].strip().replace('~','').replace(',','')
if(reached[-1] == 'k'):
    reached = float(reached.replace('k',''))
    reached = reached * 1000
elif(reached[-1] == 'm'):
    reached = float(reached.replace('m',''))
    reached = reached * 1000000
else:
    reached = float(reached)
print(reached)

#1593077、9273190
profile = user_page.find('ul',class_='list-unstyled')
location = ''
age = ''
views = ''
for x in profile.find_all('li'):
    if(x.find('svg',class_='svg-icon iconLocation') is not None):#代表這傢伙有居住地
        location = x.text.strip()
    if(x.find('svg',class_='svg-icon iconHistory') is not None):#這傢伙的帳號創建日
        age = x.find('span')['title'].split('Z')[0]
    if(x.find('svg',class_='svg-icon iconEye') is not None):
        views = int(x.text.split(' p')[0].strip().replace(',',''))
        

cursor.execute('INSERT INTO User (user_id,reputation,answers,questions,reached,location,age,views) VALUES('
               + '"' + owner_id + '",' + str(reputation) + ',' + str(answers) + ',' + str(questions) + ',' + str(reached) + ',"' + location + '","' + age + '",' + str(views) + ')')
db.commit()

print(location + ' | ' , age , ' | ' , views)

448
675
2300000.0
Amsterdam, Netherlands |  2012-08-12 06:35:18  |  4076


In [68]:
page = 1
total_count = 0
total_score = 0
while True:
    toptags_url = 'https://api.stackexchange.com/2.2/users/'+owner_id+'/top-tags?page=' + str(page) + '&pagesize=100&site=stackoverflow'
    user_toptags = bs(requests.get(toptags_url).text,'html.parser')
    json_str = json.loads(user_toptags.text)

    cursor.execute('SELECT id FROM User WHERE user_id = '+str(owner_id))
    user_index = cursor.fetchall()#user_index[0][0]是int

    for x in range(len(json_str['items'])):
        total_count = json_str['items'][x]['answer_count'] + json_str['items'][x]['question_count']#把answer和question的數量加起來
        total_score = json_str['items'][x]['answer_score'] + json_str['items'][x]['question_score']#把answer和question的分數加起來
        average = total_score/total_count#算平均每個問題得到幾分

        cursor.execute('SELECT id FROM Tags WHERE name = "'+str(json_str['items'][x]['tag_name'])+'"')
        tag_id = cursor.fetchall()
        if (len(tag_id) != 0):#代表這個tag存在於Tags這個table裡面
            tag_id = tag_id[0][0]
        else:
            tag_id = 0
        cursor.execute('INSERT INTO UserTag (user_index,user_id,tag_id,answer_count,answer_score,question_count,question_score,total_count,total_score,average) VALUES('
                           +str(user_index[0][0])+','+str(owner_id)+','+str(tag_id)+','+str(json_str['items'][x]['answer_count'])+','+str(json_str['items'][x]['answer_score'])+','+str(json_str['items'][x]['question_count'])+','+str(json_str['items'][x]['question_score'])+','+str(total_count)+','+str(total_score)+','+str(average)+')')
        db.commit()

    if (json_str['has_more'] is False):#還沒結束，後面還有
        break
    page = page + 1

In [70]:
cursor.close()
db.close()

Error: Already closed